# Module 5: Road Sign Detection

While being able to detect AprilTags are very useful, there aren't AprilTags located beneath road signs on actual roads. Therefore, we want to use the concepts of what we applied in Module 4, along with the computer vision techniques we learned in Module 3, to detect the road signs by looking at both their shape and color.

This module should follow Module 4: AprilTags

You know the drill.

In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
from IPython.display import display, Image, clear_output
import time

robot = Robot()
camera = Camera.instance()

In [ ]:
while True:

    clear_output(wait = True)
    orig_image = np.array(camera.value)
    image = orig_image

    # Convert to HSV (Hue, Saturation, Value) color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # TODO: Change the values of these.
    # Define range of red color in HSV space
    lower_red1 = np.array([0, 120, 70])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])

    # Create a mask for red color
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = mask1 | mask2

    # Apply the mask to the image
    red_traffic_signs = cv2.bitwise_and(image, image, mask=mask)

    # Dilate image to get rid of white stop
    dilated_image = cv2.dilate(red_traffic_signs, np.ones((5, 5), np.uint8), iterations = 2)
    eroded_image = cv2.erode(dilated_image, np.ones((5, 5), np.uint8))

    # # Define the sharpening kernel
    # kernel = np.array([[-1, -1, -1],
    #                 [-1, 9, -1],
    #                 [-1, -1, -1]])

    # # Apply the kernel to the image using filter2D
    # sharpened_image = cv2.filter2D(eroded_image, -1, kernel)

    # Convert to grayscale
    gray = cv2.cvtColor(dilated_image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detect edges using Canny edge detector
    edges = cv2.Canny(blurred, 100, 200)

    # Find contours
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Minimum area we look at
    min_area = 800

    # Loop through contours
    for contour in contours:
        contour_area = cv2.contourArea(contour)
        if contour_area > min_area:
            # Approximate the contour to a polygon
            epsilon = 0.01 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # If the shape has 8 vertices, it could be a stop sign (octagon)
            if len(approx) == 8:
                cv2.drawContours(orig_image, [approx], -1, (0, 255, 0), 2)
                cv2.putText(orig_image, 'Stop Sign', tuple(approx[0][0]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                cv2.putText(orig_image, str(contour_area), (approx[0][0][0], approx[0][0][1] + 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    display(Image(data=bgr8_to_jpeg(orig_image)))
    time.sleep(0.05)
    